In [ ]:
%matplotlib inline
from fastai.vision.all import *
import sklearn
from fer import *

In [ ]:
from matplotlib import ticker

In [ ]:
setMatPlotLib(style='latex')

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
USE_GPUS = '0,1,2,3'
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = USE_GPUS
N_GPUS = len(USE_GPUS.split(','))

In [ ]:
def str_formatter(string, pp, cycle):
    pp.text(string)

plain = get_ipython().display_formatter.formatters['text/plain']
plain.for_type(str, str_formatter)

In [ ]:
FIGURES_DATA_PATH = f'{PROCESSED_PATH}/data/figures_affectnet'

In [ ]:
vocab=['angry', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

basedataset = 'affectnet'

## Setup

In [ ]:
import torch
import fastai
print(torch.__version__)
print(fastai.__version__)

## Dataset analysis

In [ ]:
ds = FERDataset2(basedataset, vocab=vocab)
df = ds.getAggregatedCSV()

In [ ]:
counts = dict([(partition, df[df['partition']==partition]['race'].value_counts(sort=True, normalize=True)) for partition in ['train', 'val']])
counts = pd.DataFrame(counts).sort_values(by = 'train', ascending=False)

counts.index = counts.index.str.replace(' ', '\n')
counts.index = counts.index.str.replace('_', '\n')

counts.index = [''.join([c for c in name if c.isupper()]) for name in counts.index]

counts.plot.bar(figsize=(3.4, 1.5),
                  colormap = 'Accent')
plt.xticks(range(len(counts.index)), counts.index, rotation=0)
plt.ylabel('Data proportion')
plt.show()

plt.tight_layout()
plt.savefig(FIGURES_DATA_PATH + '/source_race_distribution.pgf')

In [ ]:
counts

In [ ]:
counts = {'train': df[df['partition']=='train']['gender'].value_counts(sort=True, normalize=True),
          'val': df[df['partition']=='val']['gender'].value_counts(sort=True, normalize=True)}
counts = pd.DataFrame(counts).sort_values(by = 'train')
counts

In [ ]:
counts = dict([(emotion, df[(df['partition']=='train') & (df['label']==emotion)]['gender'].value_counts(normalize=True)) for emotion in vocab])
counts = pd.DataFrame(counts).T
counts['diff'] = counts['Female'] - counts['Male']
counts = counts.sort_values(by = 'diff')
counts = counts.drop('diff', axis=1)

axs = counts.plot.bar(figsize=(3.4, 2),
                  colormap = 'Accent')
plt.ylabel('Train dataset proportion')
plt.legend(loc='lower right')

plt.show()

plt.tight_layout()
plt.savefig(FIGURES_DATA_PATH + '/source_gender_distribution.pgf')

In [ ]:
counts

In [ ]:
def representationalBiasSummary(df, targets=['age', 'gender', 'race'], partition = 'train'):
    res = {}
    for target in targets:
        bias, groups = representationalBias(df, target, partition)
        res[f'{target}-groups'] = groups
        res[f'{target}-bias'] = bias
    return res

def stereotypicalBiasSummary(df, targets=['age', 'gender', 'race'], partition = 'train'):
    res = {}
    for target in targets:
        res[f'{target}-nmi'] = nmi(df[df['partition']==partition], 'label', target)
    return res
      
table = {}
targets = ['gender', 'race']

datasets = {
    'Original': basedataset,
    'Race balanced': f'compdataset_{basedataset}_race_balanced',
    'Gender balanced': f'compdataset_{basedataset}_gender_balanced',
    'Gender biased (M)': f'compdataset_{basedataset}_gender_biased_Male-only',
    'Gender biased (F)': f'compdataset_{basedataset}_gender_biased_Female-only',
}

for key, ds in datasets.items():
    df = FERDataset2(ds, vocab=vocab, load=False).getAggregatedCSV()
    table[key] = representationalBiasSummary(df, targets=targets)
    table[key] = table[key] | stereotypicalBiasSummary(df, targets=targets)

table = pd.DataFrame(table).T
for target in targets:
    table = table.astype({f"{target}-groups": int})
table = table[[
    'race-groups',
    'race-bias',
    'race-nmi',
    'gender-groups',
    'gender-bias',
    'gender-nmi',
]]
display(table)

In [ ]:
t2 = table.copy()
# t2 = t2.reset_index()
t2['race-groups'] = t2['race-groups'].apply(lambda x: f"${x}$")
t2['gender-groups'] = t2['gender-groups'].apply(lambda x: f"${x}$")
t2['race-bias'] = t2['race-bias'].apply(lambda x: f"{x:.4f}" if x>1e-10 else f"$0$")
t2['gender-bias'] = t2['gender-bias'].apply(lambda x: f"{x:.4f}" if x>1e-10 else f"$0$")
t2['race-nmi'] = t2['race-nmi'].apply(lambda x: f"{x:.4f}" if x>1e-10 else f"$0$")
t2['gender-nmi'] = t2['gender-nmi'].apply(lambda x: f"{x:.4f}" if x>1e-10 else f"$0$")

t2.insert(loc=0, column='Train data 1', value=['', 'Balanced', '', 'Gender biased', ''])
t2.insert(loc=1, column='Train data 2', value=['Original', 'Race', 'Gender', 'M', 'F'])

t2.columns = pd.MultiIndex.from_arrays([['', '', *['Race bias']*3, *['Gender bias']*3], ['Dataset', 'Dataset'] + ['Groups', 'NDS', 'NMI'] * 2])
display(t2)
t2.to_latex(index=False, escape=False)


In [ ]:
t2 = table[[
    'race-bias',
    'gender-bias',
    'race-nmi',
    'gender-nmi',
]].copy()
# t2 = t2.reset_index()
t2['race-bias'] = t2['race-bias'].apply(lambda x: f"${x:.4f}$")
t2['gender-bias'] = t2['gender-bias'].apply(lambda x: f"${x:.4f}$")
t2['race-nmi'] = t2['race-nmi'].apply(lambda x: f"${x:.4f}$")
t2['gender-nmi'] = t2['gender-nmi'].apply(lambda x: f"${x:.4f}$")

t2.insert(loc=0, column='Train data 1', value=['Original', 'Balanced', '', 'Gender biased', ''])
t2.insert(loc=1, column='Train data 2', value=['', 'Race', 'Gender', 'M', 'F'])

t2.columns = pd.MultiIndex.from_arrays([[' ', ' ', 
                                         *['Representational bias (NSD)']*2, 
                                         *['Stereotypical bias (NMI)']*2], 
                                        ['Dataset', ' '] + ['Race (7)', 'Gender (2)'] * 2])
display(t2)
t2.to_latex(index=False, escape=False)


In [ ]:
def plotNPMIMatrixes(matrixes):
    fig, axes = plt.subplots(ncols=len(matrixes), 
                             figsize=(3.2, 3),
                             gridspec_kw={'width_ratios': [len(m.columns) for m in matrixes]},
                             sharey=True)

    for matrix, ax in zip(matrixes, axes):
        if matrix.columns.name == 'gender':
            matrix.columns = matrix.columns.str.replace('Female', 'F')
            matrix.columns = matrix.columns.str.replace('Male', 'M')
    
        if matrix.columns.name == 'race':
            matrix.columns = matrix.columns.str.replace('Black', 'B')
            matrix.columns = matrix.columns.str.replace('East Asian', 'EA')
            matrix.columns = matrix.columns.str.replace('Indian', 'I')
            matrix.columns = matrix.columns.str.replace('Latino_Hispanic', 'LH')
            matrix.columns = matrix.columns.str.replace('Middle Eastern', 'ME')
            matrix.columns = matrix.columns.str.replace('Southeast Asian', 'SA')
            matrix.columns = matrix.columns.str.replace('White', 'W')
            matrix = matrix[['W', 'LH', 'ME', 'B', 'EA', 'I', 'SA']]
            
        ms = ax.matshow(matrix, cmap='RdBu', vmin = -0.2, vmax = 0.2)
        ax.tick_params(axis="x", bottom=True, top=False, labelbottom=True, labeltop=False)
        ax.set_xticks(range(len(matrix.columns)), labels=matrix.columns)
        if ax == axes[0]:
            ax.set_yticks(range(len(matrix)), labels=matrix.index)
        else:
            ax.tick_params(axis="y", left=False)

        for (i,j), z in np.ndenumerate(matrix):
            ax.text(j, i, f'{z:.2f}', ha="center", va="center", fontsize='xx-small', c='w' if abs(z)>0.1 else 'k')
                
    
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.3, 0.02, 0.4])
    fig.colorbar(ms, cax=cbar_ax)
    fig.show()

df = FERDataset2(basedataset, vocab=vocab, load=False).getAggregatedCSV()

targets = ['gender', 'race']
mats = []
for target in targets:
    n = npmi(df, 'label', target)
    mats.append(n)
plotNPMIMatrixes(mats)

fig = plt.gcf()
plt.savefig(FIGURES_DATA_PATH + '/source_npmi.pgf', bbox_inches='tight')

## Model analysis

In [ ]:
exp = 'exp_affectnet'

In [ ]:
print(f'Exp: {exp}')
SAVE_RESULTS = f'{RESULTS_PATH}/{exp}.pkl'
results = pickle.load(open(SAVE_RESULTS, 'rb'))
print(f'Experiment file [{exp}.pkl] loaded correctly')

In [ ]:
calculated_metrics = None

In [ ]:
summaries, calc_metrics = bias_analysis(results, 
                              train_names=results['datasets_train'], 
                              test_names=[basedataset],
                              targets=['race', 'gender'],
                              measure = privilege_measurement,
                              vocab=vocab)

In [ ]:
resultsDF = generateResultsDF(results, vocab=vocab)
stats = {}
for train in resultsDF['train'].unique():
    filtered = resultsDF[resultsDF['train'] == train]
    stats_train = {}
    for label in vocab:
        for gender in ['Male', 'Female']:
            filtered_label = filtered[(filtered['l_true'] == label) & (filtered['gender'] == gender)]
            recall = (filtered_label['l_pred'] == label).sum() / len(filtered_label)
            stats_train[(label, gender)] = recall
        stats_train[(label, 'Diff')] = abs(stats_train[(label, 'Male')] - stats_train[(label, 'Female')])
    stats[train] = stats_train
statsDF = pd.DataFrame(stats).T
displayDF(statsDF * 100, style='global')

In [ ]:
# Gráfica unificada

x = summaries[basedataset]['race']['size']
y0 = summaries[basedataset]['race']['accuracy_mean']
y0_e = summaries[basedataset]['race']['accuracy_std']
y1 = summaries[basedataset]['race']['metric_mean']
y1_e = summaries[basedataset]['race']['metric_std']
y2 = summaries[basedataset]['gender']['metric_mean']
y2_e = summaries[basedataset]['gender']['metric_std']


partials = 10

labels = summaries[basedataset]['gender'].index

labels = labels.str.replace(f'{basedataset}_sub_', 'Original / ')
labels = labels.str.replace('/ 1\.0', '', regex=True)
labels = labels.str.replace('compdataset_', '', regex=True)
labels = labels.str.replace('_', ' ')
labels = labels.str.capitalize()
labels = labels.str.replace('female-only', '(F)')
labels = labels.str.replace('male-only', '(M)')
labels = labels.str.replace('Ferplus', 'FER+')

cmap = plt.get_cmap('Dark2')

fig, axes = plt.subplots(3, 1, figsize=(3.4, 3.6), sharex = True)
ax1 = axes.flatten()[0]
ax2 = axes.flatten()[1]
ax3 = axes.flatten()[2]

linewidth = 0.8
alpha = 0.3

l1 = ax1.plot(x[:partials], y0[:partials], label='Stratified subsets', c=cmap(7), linewidth=linewidth)
ax1.fill_between(x[:partials], y0[:partials]-y0_e[:partials], y0[:partials]+y0_e[:partials], 
                 color=cmap(7), alpha=alpha, linewidth=0)

l2 = ax2.plot(x[:partials], y1[:partials], label='Stratified subsets', c=cmap(7), linewidth=linewidth)
ax2.fill_between(x[:partials], y1[:partials]-y1_e[:partials], y1[:partials]+y1_e[:partials], 
                 color=cmap(7), alpha=alpha, linewidth=0)

l3 = ax3.plot(x[:partials], y2[:partials], label='Stratified subsets', c=cmap(7), linewidth=linewidth)
ax3.fill_between(x[:partials], y2[:partials]-y2_e[:partials], y2[:partials]+y2_e[:partials], 
                 color=cmap(7), alpha=alpha, linewidth=0)

elinewidth = 0.8
markersize = 4
markersize2 = 16
# capsize = 0.4
capsize = 2

i = partials
marker = (3, 0, 0)
p1 = ax1.errorbar(x[i], y0[i], yerr=y0_e[i], marker=None, markersize=0, label=labels[i], color=cmap(0), linewidth=elinewidth, capsize=capsize)
p2 = ax2.errorbar(x[i], y1[i], yerr=y1_e[i], marker=marker, markersize=markersize, label=labels[i], color=cmap(0), linewidth=elinewidth, capsize=capsize)
p1 = ax1.scatter(x[i], y0[i], [markersize2], marker=marker, label=labels[i], color=cmap(0), linewidths=1.0)

i = partials +3
marker = (6, 0, 0)
p5 = ax1.errorbar(x[i], y0[i], yerr=y0_e[i], marker=marker, markersize=markersize, label=labels[i], color=cmap(2), linewidth=elinewidth, capsize=capsize)
p6 = ax3.errorbar(x[i], y2[i], yerr=y2_e[i], marker=marker, markersize=markersize, label=labels[i], color=cmap(2), linewidth=elinewidth, capsize=capsize)
p5 = ax1.scatter(x[i], y0[i], [markersize2], marker=marker, label=labels[i], color=cmap(2), linewidths=1.0)

i = partials +2
marker = (5, 0, 0)
p3 = ax1.errorbar(x[i], y0[i], yerr=y0_e[i], marker=marker, markersize=markersize, label=labels[i], color=cmap(3), linewidth=elinewidth, capsize=capsize)
p4 = ax3.errorbar(x[i], y2[i], yerr=y2_e[i], marker=marker, markersize=markersize, label=labels[i], color=cmap(3), linewidth=elinewidth, capsize=capsize)
p3 = ax1.scatter(x[i], y0[i], [markersize2], marker=marker, label=labels[i], color=cmap(3), linewidths=1.0)

i = partials +1
marker = (4, 0, 0)
p1b = ax1.errorbar(x[i], y0[i], yerr=y0_e[i], marker=marker, markersize=markersize, label=labels[i], color=cmap(5), linewidth=elinewidth, capsize=capsize)
p2b = ax3.errorbar(x[i], y2[i], yerr=y2_e[i], marker=marker, markersize=markersize, label=labels[i], color=cmap(5), linewidth=elinewidth, capsize=capsize)
p1b = ax1.scatter(x[i], y0[i], [markersize2], marker=marker, label=labels[i], color=cmap(5), linewidths=1.0)


# ax1.legend(fontsize='x-small')
fig.legend(handles=[*l1, p1, p1b, p3, p5], fontsize='x-small', 
           loc='center right',
           bbox_to_anchor=[0, 0.2, 1, 0.8],
           framealpha=1.0,
           markerscale=1.3)

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)

ax3.set_xlabel('Training size')
ax1.set_ylabel('Accuracy')
ax2.set_ylabel('Model racial bias')
ax3.set_ylabel('Model gender bias')
ax3.xaxis.set_major_formatter("{x:,.0f}")
plt.yticks(fontsize='x-small')
plt.show()
# plt.tight_layout()
plt.savefig(FIGURES_DATA_PATH + '/model_racialgender.pgf', bbox_inches='tight')

In [ ]:
composed = summaries[basedataset]['race'][['size']].copy()
composed['size'] = composed['size'].apply(lambda x: f"{int(x):,}")
composed.rename(columns = {'size':'Size'}, inplace = True)
composed['Accuracy'] = ("$" + 
                        (summaries[basedataset]['race']['accuracy_mean'] * 100).apply(lambda x: f"{x:.1f}") + 
                        "\pm" + 
                        (summaries[basedataset]['race']['accuracy_std'] * 100).apply(lambda x: f"{x:.1f}") +
                        "$")
composed['Race bias'] = ("$" + 
                           (summaries[basedataset]['race']['metric_mean']).apply(lambda x: f"{x:.3f}") + 
                           "\pm" + 
                           (summaries[basedataset]['race']['metric_std']).apply(lambda x: f"{x:.3f}") +
                           "$")
composed['Gender bias'] = ("$" + 
                           (summaries[basedataset]['gender']['metric_mean']).apply(lambda x: f"{x:.3f}") + 
                           "\pm" + 
                           (summaries[basedataset]['gender']['metric_std']).apply(lambda x: f"{x:.3f}") +
                           "$")

datasets = list(composed.index)
datasets1 = [''] * len(datasets)
datasets2 = [''] * len(datasets)
datasets1[0] = 'Original'
datasets2[:10] = [f'{float(d.split("_")[2]) * 100:.0f}\%' for d in datasets[:10]]
datasets1[10] = 'Balanced'
datasets2[10] = 'Race'
datasets2[11] = 'Gender'
datasets1[12] = 'Gender biased'
datasets2[12] = 'M'
datasets2[13] = 'F'

composed.insert(loc=0, column='Train data 1', value=datasets1)
composed.insert(loc=1, column='Train data 2', value=datasets2)

composed.columns = pd.MultiIndex.from_arrays([['', '', '', '', 'Bias', 'Bias'], ['Train data', 'Train data', 'Size', 'Accuracy', 'Race OD', 'Gender OD']])
display(composed)
composed.to_latex(index=False, escape=False)